In [7]:
import numpy as np
from scipy.optimize import minimize
import csv
import random


In [131]:
parkingLotSize = 173
chargingProbability = 0.25
installCost = 5000
rebatePercent = 0.7
chargeCharge = 1.5
weeks = 522 #10 years

In [41]:
# Open the CSV file
csv_file = open('time_demand.csv', 'r')
csv_file2 = open('vehicle_registration_2022.csv', 'r')

# Create a csv.reader object
csv_reader = csv.reader(csv_file)
csv_reader2 = csv.reader(csv_file2)

# Initialize an empty array to store the CSV data
data_array = []
data_array2 = []

# Read and store data row by row
for row in csv_reader:
    data_array.append(row)
for row in csv_reader2:
    data_array2.append(row)

# Close the CSV file
csv_file.close()
csv_file2.close()

demandHourly = [item for sublist in [row[1:] for row in data_array[1:-1]] for item in sublist]
vehicleRegistration2022 = data_array2

In [5]:
#Need a script that calculates percent of vehicles in area that are EVs based on vehicle registration data
#Defines variable percentEV

# Specify the column index you want to analyze
column_index = 3  # 3 corresponds to EV included
target_text = "Yes"
count_matches = 0

# Iterate through the rows and check for matches
for row in vehicleRegistration2022[1:]:  # Skip the header row
    if row[column_index] == target_text:
        count_matches += 1

# Calculate the percentage
total_rows = len(vehicleRegistration2022) - 1  # Subtract 1 to exclude the header row
percentEV = count_matches / total_rows
percentEV

0.03045657988467897

In [81]:
def chargingDemand(demandHourly,percentEV,chargingProbability,chargers,parkingLotSize):
    #Takes inputs: array demandHourly which is 13 (hours per day) by 7 (days per week), float percentEV, float chargingProbability
    #Returns total number of charging instances per week weeklyChargeCount
    weeklyChargeCount = 0

    for i in demandHourly:
        chargersOpen = chargers
        spotsCount = parkingLotSize
        while spotsCount > 0:
            EVprobability = float(i)*percentEV*chargingProbability*1000
            random_number = random.randint(1, 1000)
            if random_number < EVprobability and chargersOpen > 0:
                weeklyChargeCount += 1
                chargersOpen -= 1
            spotsCount -= 1
            
    return(weeklyChargeCount)

In [84]:
chargeHours = chargingDemand(demandHourly,percentEV,chargingProbability,1,parkingLotSize)
print(chargeHours)

39


In [114]:
def profit(installCost,rebatePercent,chargeCharge, weeks, demandHourly,percentEV,chargingProbability,chargers,parkingLotSize):
    #Takes inputs: float installCost in CAD, float rebatePercent, chargeCharge in CAD/hour (change this eventually to CAD/kWh and weeklyChargeCount to kwH/week)
    #plus params for ChargingDemand
    weeklyChargeCount = chargingDemand(demandHourly,percentEV,chargingProbability,chargers,parkingLotSize)
    costs = installCost*(1-rebatePercent)*chargers
    #This needs to be adjusted for inflation later
    revenue = weeklyChargeCount*chargeCharge*weeks
    return(revenue-costs)
    #Returns total profit/loss in CAD

In [ ]:
#Lifecycle analysis later

In [ ]:
#Battery modelling later (this might go in the script up top actually)

In [129]:
def chargerSizing(installCost,rebatePercent,chargeCharge,weeks,demandHourly,percentEV,chargingProbability):
    #This is an optimization function that finds the optimal number of chargers to maximize profit
    defaultChargers = 1
    bnds = [(1,91)]
    result = minimize(lambda x: -profit(installCost,rebatePercent,chargeCharge, weeks, demandHourly,percentEV,chargingProbability,x,parkingLotSize), defaultChargers, bounds=bnds)
    chargers = result.x[0]
    netprofit = -result.fun
    print("The optimal number of chargers is")
    print(chargers)
    print("The profit will be")
    print(netprofit)
    return(chargers)

In [132]:
chargerSizing(installCost,rebatePercent,chargeCharge,weeks,demandHourly,percentEV,chargingProbability)

The optimal number of chargers is
1.0000075158478308
The profit will be
3065.988726228254


1.0000075158478308